In [1]:
%cd /Users/georgi/dev/dialogue_modeling

/Users/georgi/dev/dialogue_modeling


In [2]:
from supporting_classifiers.agreement_classifier import *
from solution_tracker.simple_sol import solution_tracker, process_raw_to_solution_tracker

In [3]:
import spacy
import string
from read_data import read_solution_annotaions, read_wason_dump, read_3_lvl_annotation_file
import pandas as pd
from featurisers.raw_wason_featuriser import get_y

In [4]:
nlp = spacy.load("en_core_web_sm")

In [5]:
hierch_data = read_3_lvl_annotation_file('3lvl_anns.tsv')


for a in hierch_data:
    a.preprocess_everything(nlp)

raw_data = read_wason_dump('data/all/')

conversations_to_process = {}
for ann in hierch_data:
    raw = [r for r in raw_data if r.identifier == ann.identifier][0]
    ann.raw_db_conversation = raw.raw_db_conversation
    conversations_to_process[ann.identifier] = ann

In [6]:
import pandas as pd
import plotly.express as px

In [8]:
from solution_tracker.augment_with_solution import merge_with_solution_raw
import plotly.graph_objects as go
from featurisers.raw_wason_featuriser import calculate_stats, preprocess_conversation_dump

In [9]:
def process_in_window(conversation, threshold, window_size, bad_cases=False): 
    output = []
    for index, message in enumerate(messages_supervised):
        begining = 0 if index-window_size < 0 else index-window_size
        running_window = messages_supervised[begining:index]
        if len(running_window) == 0:
            continue
        diff = running_window[-1].annotation['team_performance'] - running_window[0].annotation['team_performance']
        if bad_cases:
            if diff > threshold:
                continue
        else:
            if diff < threshold:
                continue
                
        inner_window = []
        for item in running_window:
            inner_window.append({'window_improvement': diff ,'message': item.content, 'annotation': item.annotation,
                          'improvement': message.annotation['performance_change']})
        output.append(inner_window)
      
    return output

In [11]:
good_cases = []
bad_cases = []
for key, conv in conversations_to_process.items():
    messages_supervised = merge_with_solution_raw(conv)
    output_good = process_in_window(messages_supervised, 0.1, 6)
    output_bad = process_in_window(messages_supervised, -0.1, 6, True)

    good_cases.extend(output_good)
    bad_cases.extend(output_bad)


In [12]:
len(good_cases)

211

In [13]:
len(bad_cases)

105

In [14]:
orderred_good = sorted(good_cases, key=lambda k: k[0]['window_improvement'], reverse=True) 


In [15]:
orderred_good

[[{'window_improvement': 0.5,
   'message': 'yeah but that could be a fluke so i think to be more sure, two cards would be more accurate',
   'annotation': {'additional': {'0'},
    'type': 'Non-probing-deliberation',
    'target': 'Reasoning',
    'Alpaca': {'6', 'A'},
    'Bat': {'6', 'A'},
    'team_performance': 0.5,
    'performance_change': 0.0},
   'improvement': 0.0},
  {'window_improvement': 0.5,
   'message': 'Alright',
   'annotation': {'additional': {'0'},
    'type': 'Non-probing-deliberation',
    'target': 'Agree',
    'Alpaca': {'6', 'A'},
    'Bat': {'6', 'A'},
    'team_performance': 0.5,
    'performance_change': 0.0},
   'improvement': 0.0},
  {'window_improvement': 0.5,
   'message': 'So A and 7?',
   'annotation': {'additional': {'solution_summary'},
    'type': 'Probing',
    'target': 'Solution',
    'Alpaca': {'6', 'A'},
    'Bat': {'7', 'A'},
    'team_performance': 0.75,
    'performance_change': 0.25},
   'improvement': 0.0},
  {'window_improvement': 0.5,
  

In [16]:
def print_orderred(orderd):
    for item in orderd:
        for message in item:
            print("{}\t{}\t{}".format(message['message'], message['window_improvement'], message['annotation']))
        print('~~~~~~~~~~~~~\t~~~~~~~~~~\t~~~~~~~~~~~')

In [17]:
print_orderred(orderred_good)

yeah but that could be a fluke so i think to be more sure, two cards would be more accurate	0.5	{'additional': {'0'}, 'type': 'Non-probing-deliberation', 'target': 'Reasoning', 'Alpaca': {'6', 'A'}, 'Bat': {'6', 'A'}, 'team_performance': 0.5, 'performance_change': 0.0}
Alright	0.5	{'additional': {'0'}, 'type': 'Non-probing-deliberation', 'target': 'Agree', 'Alpaca': {'6', 'A'}, 'Bat': {'6', 'A'}, 'team_performance': 0.5, 'performance_change': 0.0}
So A and 7?	0.5	{'additional': {'solution_summary'}, 'type': 'Probing', 'target': 'Solution', 'Alpaca': {'6', 'A'}, 'Bat': {'7', 'A'}, 'team_performance': 0.75, 'performance_change': 0.25}
that's what i'm thinking but it's just as likely to be A and 6 so which do you think is more likely?	0.5	{'additional': {'0'}, 'type': 'Non-probing-deliberation', 'target': 'Solution', 'Alpaca': {'6', 'A'}, 'Bat': {'7', 'A'}, 'team_performance': 0.75, 'performance_change': 0.0}
It's a bit hard because both can work	0.5	{'additional': {'0'}, 'type': 'Non-pro

In [22]:
good_probing = [] 

for conv in orderred_good:
    for message in conv:
        if 'type' in message['annotation'] and message['annotation']['type'] == 'Probing':
            good_probing.append(message)

In [24]:
good_probing

[{'window_improvement': 0.5,
  'message': 'So A and 7?',
  'annotation': {'additional': {'solution_summary'},
   'type': 'Probing',
   'target': 'Solution',
   'Alpaca': {'6', 'A'},
   'Bat': {'7', 'A'},
   'team_performance': 0.75,
   'performance_change': 0.25},
  'improvement': 0.0},
 {'window_improvement': 0.5,
  'message': 'So A and 7?',
  'annotation': {'additional': {'solution_summary'},
   'type': 'Probing',
   'target': 'Solution',
   'Alpaca': {'6', 'A'},
   'Bat': {'7', 'A'},
   'team_performance': 0.75,
   'performance_change': 0.25},
  'improvement': 0.0},
 {'window_improvement': 0.43700000000000006,
  'message': '@Alpaca why do you think you need to turn s?',
  'annotation': {'additional': {'specific_addressee'},
   'type': 'Probing',
   'target': 'Reasoning',
   'Beaver': {'6', '7', 'A'},
   'Leopard': {'6', 'S'},
   'Alpaca': {'6', 'S'},
   'Tiger': {'6', '7', 'A'},
   'team_performance': 0.375,
   'performance_change': 0.0},
  'improvement': 0.0},
 {'window_improvement

In [ ]:
len(good_probing)

In [19]:
orderred_good_probing = sorted(good_probing, key=lambda k: k['improvement'], reverse=True) 


NameError: name 'good_probing' is not defined

In [ ]:
orderred_good_probing

In [ ]:
print_orderred(bad_cases)

In [ ]:
orderred_bad = sorted(bad_cases, key=lambda k: k[0]['improvement'], reverse=False) 


In [ ]:
print_orderred(orderred_bad)

In [ ]:
orderred_bad

In [ ]:
bad_probing = [g for g in bad_cases if g['annotation']['type'] == 'Probing']

In [ ]:
print_orderred(bad_probing)

In [ ]:
orderred_bad_probing = sorted(bad_probing, key=lambda k: k['improvement'], reverse=False) 
orderred_bad_probing